In [1]:
# importing libraries 
import pandas as pd
import numpy as np
import os
import re
import string
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re
import warnings
warnings.filterwarnings("ignore")

In [2]:
# loading the testing and training datasets
CovidMask1=pd.read_csv('CovidMaskPeriod1Raw.csv')
CovidMask2=pd.read_csv('CovidMaskPeriod2Raw.csv')
CovidMaskHydrated=pd.read_csv('CovidMaskTrainingDatasetRaw.csv')

In [5]:
CovidMaskHydrated.shape

(6292091, 36)

In [3]:
CovidMask1.shape

(1753215, 11)

In [4]:
CovidMask2.shape

(1095292, 11)

In [6]:
CovidMaskHydrated.head()

,coordinates,created_at,hashtags,media,urls,favorite_count,id,in_reply_to_screen_name,in_reply_to_status_id,in_reply_to_user_id,...,user_friends_count,user_listed_count,user_location,user_name,user_screen_name,user_statuses_count,user_time_zone,user_urls,user_verified,Sentiment
0,NaN,Tue Jun 30 04:25:57 +0000 2020,NaN,NaN,NaN,0,1277820648123969536,NaN,NaN,NaN,...,47.0,0.0,"Panama City Beach, FL",Andrew Brissette,abrissette94,60.0,NaN,NaN,False,0.107143
1,NaN,Tue Jun 30 04:25:57 +0000 2020,NaN,NaN,https://twitter.com/benandjerrys/status/127728...,2,1277820645041147904,NaN,NaN,NaN,...,365.0,0.0,NaN,Christa Squire,Christa_Vi,4460.0,NaN,NaN,False,0.000000
2,NaN,Tue Jun 30 04:25:58 +0000 2020,NaN,NaN,https://twitter.com/i/events/1277716877960073216,0,1277820650783215616,NaN,NaN,NaN,...,39.0,0.0,NaN,Ali Khan,AliKhan40489666,21.0,NaN,NaN,False,0.000000
3,NaN,Tue Jun 30 04:25:58 +0000 2020,NaN,https://twitter.com/QuickTake/status/127768814...,NaN,0,1277820649990270976,NaN,NaN,NaN,...,963.0,1.0,"San Diego, CA",Juan Gonzalez,Abc123juice,104609.0,NaN,NaN,False,-0.500000
4,NaN,Tue Jun 30 04:25:58 +0000 2020,NaN,NaN,NaN,0,1277820650304835584,NaN,NaN,NaN,...,462.0,0.0,ทอดไข่อยู่🍳,🪸🫧mu✿hyun,bebehyunn,429830.0,NaN,https://youtu.be/2IkoKhr6Tss,False,0.000000


In [7]:
CovidMask1.head()

,id,text,created_at,author_id,in_reply_to_user_id,lang,author,entities,referenced_tweets,media,geo
0,1302033644282941440,Wear a mask or I will assault you!! (British f...,2020-09-04T23:59:46.000Z,1.447380e+08,NaN,NaN,"{""description"":""#BreakingNews; #Politics; #Eco...",NaN,NaN,NaN,NaN
1,1302033508181970950,@carole_wmajors @SpeakerPelosi @a_fly_guy What...,2020-09-04T23:59:13.000Z,1.272003e+18,NaN,NaN,"{""description"":""Wife, Mom an avid dog lover, e...",NaN,NaN,NaN,NaN
2,1302033485570473984,@Smeagle1845 @ChewyBB @jobennett78 @grantshapp...,2020-09-04T23:59:08.000Z,1.292251e+18,NaN,NaN,"{""description"":""Put a mask on! Wear your mask ...",NaN,NaN,NaN,NaN
3,1302033412006514688,Dollarama Karen calls security: INTERVIEW with...,2020-09-04T23:58:50.000Z,1.806041e+07,NaN,NaN,"{""description"":""\""The law perverted! And the p...",NaN,NaN,NaN,NaN
4,1302033369627320320,@SanjayFBPEUK @russglos @peterjukes @LesleyABe...,2020-09-04T23:58:40.000Z,2.529659e+09,NaN,NaN,"{""description"":""Grim Reaper of Football Specta...",NaN,NaN,NaN,NaN


In [8]:
CovidMask2.head()

,id,text,created_at,author_id,in_reply_to_user_id,lang,author,entities,referenced_tweets,media,geo
0,1434305238790459397,@nypost The Covid control measures have all fa...,2021-09-04T23:59:51.000Z,9.378446e+17,NaN,NaN,"{""id"":""937844576672538626"",""description"":""A ra...",NaN,NaN,NaN,NaN
1,1434305195995918337,@Cornell is proof #vaccines and masking is an ...,2021-09-04T23:59:41.000Z,1.376770e+18,NaN,NaN,"{""id"":""1376769622393819138"",""description"":""Γνῶ...",NaN,NaN,NaN,NaN
2,1434305192028086275,Fraught mask-related interactions often come a...,2021-09-04T23:59:40.000Z,1.357404e+18,NaN,NaN,"{""id"":""1357404114901037056"",""description"":""Use...",NaN,NaN,NaN,NaN
3,1434305166770012161,@OccupyDemocrats The amount of kids dying from...,2021-09-04T23:59:34.000Z,1.232516e+18,NaN,NaN,"{""id"":""1232515598573129728"",""description"":""Ame...",NaN,NaN,NaN,NaN
4,1434305150458421250,"My son wears a mask when he walks to work, som...",2021-09-04T23:59:30.000Z,1.194364e+08,NaN,NaN,"{""id"":""119436439"",""description"":""internet addi...",NaN,NaN,NaN,NaN


In [5]:
# removing unwanted columns 
CovidMask1.drop(['in_reply_to_user_id', 'lang','author_id','author','entities','referenced_tweets','media','geo'], axis=1, inplace=True)
CovidMask2.drop(['in_reply_to_user_id', 'lang','author','author_id','entities','referenced_tweets','media','geo'], axis=1, inplace=True)
CovidMaskHydrated.drop(['user_id','id','lang','coordinates', 'hashtags','media','urls','favorite_count','in_reply_to_screen_name','in_reply_to_status_id','in_reply_to_user_id','place','possibly_sensitive','quote_id','retweet_count','retweet_id','retweet_screen_name','source','tweet_url','user_created_at','user_default_profile_image','user_description','user_favourites_count','user_followers_count','user_friends_count','user_listed_count','user_location','user_name','user_screen_name','user_statuses_count','user_time_zone','user_urls','user_verified','created_at'], axis=1, inplace=True)


In [6]:
CovidMask1.head()

,id,text,created_at
0,1302033644282941440,Wear a mask or I will assault you!! (British f...,2020-09-04T23:59:46.000Z
1,1302033508181970950,@carole_wmajors @SpeakerPelosi @a_fly_guy What...,2020-09-04T23:59:13.000Z
2,1302033485570473984,@Smeagle1845 @ChewyBB @jobennett78 @grantshapp...,2020-09-04T23:59:08.000Z
3,1302033412006514688,Dollarama Karen calls security: INTERVIEW with...,2020-09-04T23:58:50.000Z
4,1302033369627320320,@SanjayFBPEUK @russglos @peterjukes @LesleyABe...,2020-09-04T23:58:40.000Z


In [7]:
CovidMask2.head()

,id,text,created_at
0,1434305238790459397,@nypost The Covid control measures have all fa...,2021-09-04T23:59:51.000Z
1,1434305195995918337,@Cornell is proof #vaccines and masking is an ...,2021-09-04T23:59:41.000Z
2,1434305192028086275,Fraught mask-related interactions often come a...,2021-09-04T23:59:40.000Z
3,1434305166770012161,@OccupyDemocrats The amount of kids dying from...,2021-09-04T23:59:34.000Z
4,1434305150458421250,"My son wears a mask when he walks to work, som...",2021-09-04T23:59:30.000Z


In [13]:
CovidMaskHydrated.head()

,text,Sentiment
0,A reminder that festivals and large concerts a...,0.107143
1,Come on Ben and Jerry’s https://t.co/RGwIaC9ytr,0.000000
2,"""Flu strain with 'human pandemic potential' fo...",0.000000
3,"WHO warns ""the worst is yet to come"" as lack o...",-0.500000
4,Work from home like a boss. Learn how Bloomber...,0.000000


In [8]:
# drop duplicates and NaN
CovidMask1.drop_duplicates(keep=False, inplace=True)
CovidMask1.dropna(inplace=True)

In [9]:
# drop duplicates and NaN
CovidMask2.drop_duplicates(keep=False, inplace=True)
CovidMask2.dropna(inplace=True)

In [7]:
# drop duplicates and NaN
CovidMaskHydrated.drop_duplicates(keep=False, inplace=True)
CovidMaskHydrated.dropna(inplace=True)

In [26]:
#duplicate = CovidMask2[CovidMask2.duplicated()]
 
#print("Duplicate Rows :")
 
#duplicate

Duplicate Rows :


,id,text,created_at,cleanText,textLemma


In [35]:
#CovidMask1.isnull().sum()

id             0
text           0
created_at    15
cleanText      0
textLemma      0
dtype: int64

In [10]:
CovidMask1.shape

(1666350, 3)

In [11]:
CovidMask2.shape

(992960, 3)

In [35]:
CovidMaskHydrated.shape

(1062834, 2)

In [ ]:
# filter the training dataset on COVID-19 and Face mask related keywords (same as testing dataset)
pattern = 'covid19.*mask|mask.*covid19|mask.*covid-19|mask.*covid-19|covid.*mask|mask.*covid|corona.*mask|mask.*corona|mask.*virus|virus.*mask|coronavirus.*mask|mask.*coronavirus|mask.*wuhancoronavirus|wuhancoronavirus.*mask|covid19.*face mask|face mask.*covid19|face mask.*covid-19|face mask.*covid-19|covid.*face mask|face mask.*covid|corona.*face mask|face mask.*corona|face mask.*virus|virus.*face mask|coronavirus.*face mask|face mask.*coronavirus|face mask.*wuhancoronavirus|wuhancoronavirus.*face mask|covid19.*face masks|face masks.*covid19|face masks.*covid-19|face masks.*covid-19|covid.*face masks|face masks.*covid|corona.*face masks|face masks.*corona|face masks.*virus|virus.*face masks|coronavirus.*face masks|face masks.*coronavirus|face masks.*wuhancoronavirus|wuhancoronavirus.*face masks|covid19.*wearamask|wearamask.*covid19|wearamask.*covid-19|wearamask.*covid-19|covid.*wearamask|wearamask.*covid|corona.*wearamask|wearamask.*corona|wearamask.*virus|virus.*wearamask|coronavirus.*wearamask|wearamask.*coronavirus|wearamask.*wuhancoronavirus|wuhancoronavirus.*wearamask|covid19.*face shield|face shield.*covid19|face shield.*covid-19|face shield.*covid-19|covid.*face shield|face shield.*covid|corona.*face shield|face shield.*corona|mask.*virus|virus.*face shield|coronavirus.*face shield|face shield.*coronavirus|face shield.*wuhancoronavirus|wuhancoronavirus.*face shield|covid19.*face cover|face cover.*covid19|face cover.*covid-19|face cover.*covid-19|covid.*face cover|face cover.*covid|corona.*face cover|face cover.*corona|face cover.*virus|virus.*face cover|coronavirus.*face cover|face cover.*coronavirus|face cover.*wuhancoronavirus|wuhancoronavirus.*face cover|covid19.*mask wearing|mask wearing.*covid19|mask wearing.*covid-19|mask wearing.*covid-19|covid.*mask wearing|mask wearing.*covid|corona.*mask wearing|mask wearing.*corona|mask wearing.*virus|virus.*mask wearing|coronavirus.*mask wearing|mask wearing.*coronavirus|mask wearing.*wuhancoronavirus|wuhancoronavirus.*mask wearing|covid19.*masking|masking.*covid19|masking.*covid-19|masking.*covid-19|covid.*masking|masking.*covid|corona.*masking|masking.*corona|masking.*virus|virus.*masking|coronavirus.*masking|masking.*coronavirus|masking.*wuhancoronavirus|wuhancoronavirus.*masking|covid19.*facial cover|facial cover.*covid19|facial cover.*covid-19|facial cover.*covid-19|covid.*facial cover|facial cover.*covid|corona.*facial cover|facial cover.*corona|facial cover.*virus|virus.*facial cover|coronavirus.*facial cover|facial cover.*coronavirus|facial cover.*wuhancoronavirus|wuhancoronavirus.*facial cover|covid19.*face shields|face shields.*covid19|face shields.*covid-19|face shields.*covid-19|covid.*face shields|face shields.*covid|corona.*face shields|face shields.*corona|face shields.*virus|virus.*face shields|coronavirus.*face shields|face shields.*coronavirus|face shields.*wuhancoronavirus|wuhancoronavirus.*face shields|mask|face mask|face masks|wearamask|masking|mask wearing|face cover|face shield|face shields|facial cover|covid|covid19|covid-19|corona|virus|coronavirus|wuhancoronavirus'
mask = CovidMaskHydrated['text'].str.contains(pattern, case=False, na=False)
CovidMaskHydrated[mask].head()


In [39]:
CovidMaskHydrated=CovidMaskHydrated[mask]

In [9]:
CovidMaskHydrated.shape

(25874, 2)

In [ ]:
# Lowercase, Removing Symbols, URL Removal, Removing Punctuation, Removing Stopwords, Removing Emoji, Tokenization and Lemmatisation
# https://www.kaggle.com/code/hassanhshah/covid-vaccine-sentiment-and-time-series-analysis Retrieved July 2022


In [12]:
# Lowercase
CovidMask1['cleanText'] = CovidMask1['text'].str.lower()

CovidMask2['cleanText'] = CovidMask2['text'].str.lower()

CovidMaskHydrated['cleanText'] = CovidMaskHydrated['text'].str.lower()



In [13]:
# Removing Symbols
CovidMask1['cleanText'] = CovidMask1['cleanText'].apply(lambda x:re.sub('\&*[amp]*\;|gt+', "", str(x)))

CovidMask2['cleanText'] = CovidMask2['cleanText'].apply(lambda x:re.sub('\&*[amp]*\;|gt+', "", str(x)))

CovidMaskHydrated['cleanText'] = CovidMaskHydrated['cleanText'].apply(lambda x:re.sub('\&*[amp]*\;|gt+', "", str(x)))



In [14]:
# URL Removal
CovidMask1['cleanText'] = CovidMask1['cleanText'].apply(lambda x:re.sub(r"http\S+", "", x))
CovidMask2['cleanText'] = CovidMask2['cleanText'].apply(lambda x:re.sub(r"http\S+", "", x))
CovidMaskHydrated['cleanText'] = CovidMaskHydrated['cleanText'].apply(lambda x:re.sub(r"http\S+", "", x))


In [15]:
# Removing Symbols
CovidMask1['cleanText'] = CovidMask1['cleanText'].apply(lambda x:re.sub("@[A-Za-z0-9_]+","", x))

CovidMask2['cleanText'] = CovidMask2['cleanText'].apply(lambda x:re.sub("@[A-Za-z0-9_]+","", x))

CovidMaskHydrated['cleanText'] = CovidMaskHydrated['cleanText'].apply(lambda x:re.sub("@[A-Za-z0-9_]+","", x))



In [16]:
# Removing Punctuation 
punRemoval = string.punctuation
def punctuationRemoval(text):
    return text.translate(str.maketrans('', '', punRemoval))


CovidMask1["cleanText"] = CovidMask1["cleanText"].apply(lambda text: punctuationRemoval(text))
CovidMask2["cleanText"] = CovidMask2["cleanText"].apply(lambda text: punctuationRemoval(text))
CovidMaskHydrated["cleanText"] = CovidMaskHydrated["cleanText"].apply(lambda text: punctuationRemoval(text))

In [17]:
# Removing Stopwords 
", ".join(stopwords.words('english'))
STOPWORDS = set(stopwords.words('english'))
def stopwordsRemoval(text):
    return " ".join([word for word in str(text).split() if word not in STOPWORDS])

CovidMask1["cleanText"] = CovidMask1["cleanText"].apply(lambda text: stopwordsRemoval(text))
CovidMask2["cleanText"] = CovidMask2["cleanText"].apply(lambda text: stopwordsRemoval(text))
CovidMaskHydrated["cleanText"] = CovidMaskHydrated["cleanText"].apply(lambda text: stopwordsRemoval(text))


In [18]:
# Removing Emoji 
def emojiRemoval(string):
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002500-\U00002BEF"  # chinese char
                               u"\U00002702-\U000027B0"
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               u"\U0001f926-\U0001f937"
                               u"\U00010000-\U0010ffff"
                               u"\u2640-\u2642"
                               u"\u2600-\u2B55"
                               u"\u200d"
                               u"\u23cf"
                               u"\u23e9"
                               u"\u231a"
                               u"\ufe0f"  # dingbats
                               u"\u3030"
                               "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', string)

CovidMask1["cleanText"] = CovidMask1["cleanText"].apply(str)
CovidMask1["cleanText"] = CovidMask1["cleanText"].apply(emojiRemoval)
CovidMask2["cleanText"] = CovidMask2["cleanText"].apply(str)
CovidMask2["cleanText"] = CovidMask2["cleanText"].apply(emojiRemoval)
CovidMaskHydrated["cleanText"] = CovidMaskHydrated["cleanText"].apply(str)
CovidMaskHydrated["cleanText"] = CovidMaskHydrated["cleanText"].apply(emojiRemoval)

In [19]:
# Tokenization and Lemmatisation
def textTokenization(text):
    text = re.split('\W+', text)
    return text
CovidMask1['token'] = CovidMask1['cleanText'].apply(lambda x: textTokenization(x.lower()))
CovidMask2['token'] = CovidMask2['cleanText'].apply(lambda x: textTokenization(x.lower()))
CovidMaskHydrated['token'] = CovidMaskHydrated['cleanText'].apply(lambda x: textTokenization(x.lower()))



wn = nltk.WordNetLemmatizer()
def textLemmatizer(text):
    text = [wn.lemmatize(word) for word in text]
    return text
CovidMask1['lemma'] = CovidMask1['token'].apply(lambda x: textLemmatizer(x))
CovidMask2['lemma'] = CovidMask2['token'].apply(lambda x: textLemmatizer(x))
CovidMaskHydrated['lemma'] = CovidMaskHydrated['token'].apply(lambda x: textLemmatizer(x))

In [20]:
# rejoin the Tokenized Lemmatisation text
# Source: https://stackoverflow.com/questions/46096363/nltk-reconstruct-sentence-from-tokens Retrieved July 2022 
def rejoin(token):
    
    return ( " ".join(token))

CovidMask1['textLemma'] = CovidMask1.apply(lambda x: rejoin(x['lemma']), axis=1)
CovidMask2['textLemma'] = CovidMask2.apply(lambda x: rejoin(x['lemma']), axis=1)
CovidMaskHydrated['textLemma'] = CovidMaskHydrated.apply(lambda x: rejoin(x['lemma']), axis=1)

In [21]:
# drop lemma and token column (unwanted columns)
CovidMask1.drop(['lemma', 'token'], axis=1, inplace=True)
CovidMask2.drop(['lemma', 'token'], axis=1, inplace=True)
CovidMaskHydrated.drop(['lemma', 'token'], axis=1, inplace=True)

In [29]:
# convert created_at to month name
CovidMask1.created_at = pd.to_datetime(CovidMask1.created_at, format='%Y-%m-%dT%H:%M:%S.000%z',errors='coerce')
CovidMask1['created_at']= pd.to_datetime(CovidMask1['created_at'])
CovidMask1['created_at']=CovidMask1['created_at'].dt.month_name().str[:3]

In [30]:
# convert created_at to month name
CovidMask2.created_at = pd.to_datetime(CovidMask2.created_at, format='%Y-%m-%dT%H:%M:%S.000%z',errors='coerce')
CovidMask2['created_at']= pd.to_datetime(CovidMask2['created_at'])
CovidMask2['created_at']=CovidMask2['created_at'].dt.month_name().str[:3]

In [42]:
# saving all data frames to CSV files
CovidMask1.to_csv('CovidMaskPeriod1Pre-processed.csv', mode='w', encoding='utf-8', index=False, line_terminator='\n')
CovidMask2.to_csv('CovidMaskPeriod2Pre-processed.csv', mode='w', encoding='utf-8', index=False, line_terminator='\n')
CovidMaskHydrated.to_csv('CovidMaskTrainingDatasetPre-processed.csv', mode='w', encoding='utf-8', index=False, line_terminator='\n')
